In [ ]:
from pyspark import SparkContext
sc = SparkContext(master='local', appName='Flight Analysis')

In [ ]:
rdd_data = sc.textFile('in/trip.txt')

In [ ]:
# Departure_Date_Time, Departure_Airport_name , Departure_Airport_Code , Destenation_Airport_name, Destenation_Airport_code, Arrival_Date_Time, airline_name, Distance, Aircraft_type, # of Seats, # of Passengers

In [ ]:
from datetime import datetime

# Extracts the year & month from a datetime string in the format YYYY-MM-DD HH:MM:SS

def date (str_date, my_format):
  return datetime.strptime(str_date, my_format).date()
def year (str_date, my_format):
  return datetime.strptime(str_date, my_format).year
def month(str_date, my_format):
  return datetime.strptime(str_date, my_format).month
def day(str_date, my_format):
  return datetime.strptime(str_date, my_format).day
def hour(str_date, my_format):
  return datetime.strptime(str_date, my_format).hour
def minute(str_date, my_format):
  return datetime.strptime(str_date, my_format).minute

###  Find the number of flights per month that depart from "McCarran International Airport" in 2024. Print the results.

In [ ]:
rdd_splited = rdd_data.map(lambda x:x.split(','))
rdd_McCarran_Airport = rdd_splited.filter(lambda x: x[1].lower()=='McCarran International Airport'.lower())
rdd_McCarran_Airport_2024 = rdd_McCarran_Airport.filter(lambda x: year(x[0],'%Y-%m-%d %H:%M:%S')==2024)
pair_rdd1 = rdd_McCarran_Airport_2024.map(lambda x: (month(x[0], '%Y-%m-%d %H:%M:%S'), 1))
flights_McCarran_Airport_2024 = pair_rdd1.reduceByKey(lambda x,y: x+y)
sorted_flights_McCarran_Airport_2024 =  flights_McCarran_Airport_2024.sortByKey()

In [ ]:
sorted_flights_McCarran_Airport_2024.collect()

[(1, 6910), (2, 6379), (3, 6874), (4, 6658), (5, 6612)]

### Find the number of passengers departing from “Frankfurt Airport” between 13:00 and 22:00. Print the results.

In [ ]:
def between_two_times(time, from_, to_):

  from_ = datetime.strptime(from_, '%H:%M').time()
  to_ = datetime.strptime(to_, '%H:%M').time()
  time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S').time()

  return from_ <= time and to_ >= time

In [ ]:
rdd_Frankfurt_Airport = rdd_splited.filter(lambda x: x[1].lower()=='Frankfurt Airport'.lower())
rdd_Frankfurt_Airport_13_22 = rdd_Frankfurt_Airport.filter(lambda x:between_two_times(x[0], '13:00', '22:00'))
rdd_Frankfurt_Airport_13_22_no_passengers =  rdd_Frankfurt_Airport_13_22.map(lambda x:int(x[-1]))

In [ ]:
rdd_Frankfurt_Airport_13_22_no_passengers.reduce(lambda x,y : x+y)

1664119

### Find all “Air France” flights that land at “Kuala Lumpur International Airport” and store the results in a file.

In [ ]:
rdd_Air_France_Kuala_Lumpur = rdd_splited.filter(lambda x: x[6].title()=='Air France' and x[3].title()=='Kuala Lumpur International Airport')
rdd_Air_France_Kuala_Lumpur.saveAsTextFile('out/Air_France_Kuala_Lumpur.txt')

In [ ]:
rdd_Air_France_Kuala_Lumpur.count()

3802

### Find the number of overnight flights in 2024 for each airline. The term "overnight flights" refers to flights that depart between 20:00 and 23:00 and arrive the next day after 03:00. Store the results in a file.

In [73]:
import calendar
def is_last_day_in_month(date):
  last_day = calendar.monthrange(date.year, date.month)[-1]
  # print(last_day,date.day)
  return last_day == date.day

In [75]:
# Test is_last_day_in_month
date_0 = datetime.strptime('2024-05-31', '%Y-%m-%d')
is_last_day_in_month(date_0)

True

In [76]:
from datetime import timedelta

def is_overnight(dep_dt,arr_dt, exp_time):
  dep_dt = datetime.strptime(dep_dt, '%Y-%m-%d %H:%M:%S')
  arr_dt = datetime.strptime(arr_dt, '%Y-%m-%d %H:%M:%S')
  exp_time = datetime.strptime(exp_time, '%H:%M').time()

  if is_last_day_in_month(dep_dt):
    return ((dep_dt.date() + timedelta(days = 31)).month == arr_dt.month) and ((dep_dt.date() + timedelta(days = 1)).day == arr_dt.day) and (arr_dt.time() >= exp_time)
  return (dep_dt.month == arr_dt.month) and ((dep_dt.date() + timedelta(days = 1)).day == arr_dt.day) and (arr_dt.time() >= exp_time)


In [77]:
# Test is_overnight
is_overnight('2024-04-30 22:11:00', '2024-05-01 4:00:00', '3:00')

True

In [78]:
rdd_flights_2024 = rdd_splited.filter(lambda x: year(x[0],'%Y-%m-%d %H:%M:%S') == 2024)
# overnight flights" refers to flights that depart between 20:00 and 23:00 and arrive the next day after 03:00.
rdd_overnight_flights_2024 = rdd_flights_2024.filter(lambda x: between_two_times(x[0], '20:00', '23:00') and is_overnight(x[0], x[5], '03:00'))

In [80]:
rdd_overnight_flights_2024.saveAsTextFile('out/overnight_flights_2024.txt')
rdd_overnight_flights_2024.count()

89450

### The occupancy rate for a flight refers to the percentage of available seats on an aircraft that are filled with passengers.  For example, if a plane has 200 seats and there are 180 passengers on board, the occupancy rate would be:

### (180 passengers / 200 seats) * 100 = 90% occupancy rate

### Task 5: Find the occupancy rate for each “Air Canada” flight in May 2024.

In [ ]:
def specific_date(flight_dt, spec_dt):
  flight_dt = datetime.strptime(flight_dt, '%Y-%m-%d %H:%M:%S')
  spec_dt = datetime.strptime(spec_dt, '%b %Y')
  return flight_dt.year == spec_dt.year and flight_dt.month == spec_dt.month

In [ ]:
# Test specific_date
specific_date('2024-04-05 06:28:33', 'Apr 2024')

True

In [ ]:
rdd_Air_Canada_May_2024 = rdd_splited.filter(lambda x :x[6].title() == 'Air Canada' and specific_date(x[0], 'May 2024'))
occupancy_rate = rdd_Air_Canada_May_2024.map(lambda x: f'{int((int(x[-1]) / int(x[-2])) * 100)} %')

In [ ]:
occupancy_rate.take(15)

['95 %',
 '93 %',
 '93 %',
 '98 %',
 '94 %',
 '93 %',
 '100 %',
 '93 %',
 '98 %',
 '96 %',
 '98 %',
 '97 %',
 '98 %',
 '96 %',
 '96 %']